# Labo 3: Spark

Authors: Christopher MEIER and Guillaume HOCHET

Based on the work of: Gary MARIGLIANO and Miguel SANTAMARIA

For MAC course given by Nastaran FATEMI

Date: November 2020

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.4`
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import $ivy.$                                  

import org.apache.spark.rdd.RDD

import org.apache.spark.sql.functions._


import org.apache.log4j.{Level, Logger}


In [2]:
// Create a spark session
// To have better integration with Jupyter, we use a wrapper class provided by almond-spark
import org.apache.spark.sql._
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
import spark.implicits._

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/12/13 12:22:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark UI

import org.apache.spark.sql._

spark: SparkSession = org.apache.spark.sql.SparkSession@728f01c3
import spark.implicits._

In [3]:
// Retrieve the Spark context
def sc = spark.sparkContext

defined function sc

In [4]:
case class Movie(id: Int, title: String, genres: Seq[String],
                 description: String, director: String, actors: Seq[String],
                 year: Int, rating: Float, votes: Int)

defined class Movie

In [5]:
  def parseRow(row: Row): Movie = {
    val id = row.getInt(0)
    val title = row.getString(1)
    val genres = row.getString(2).split(",").toList
    val description = row.getString(3)
    val director = row.getString(4)
    val actors = row.getString(5).split(",").toList
    val year = row.getInt(6)
    val rating = row.getDouble(8).toFloat
    val votes = row.getInt(9)

    Movie(id, title, genres, description, director, actors, year, rating, votes)
  }

defined function parseRow

In [6]:
val filename = "../data/IMDB-Movie-Data.csv"
val moviesDF = spark.read.format("csv")
    .option("sep", ",")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(filename)
val rddMovies = moviesDF.rdd.map(parseRow)

load at cmd5.sc:6

1 / 1

load at cmd5.sc:6

2 / 2

filename: String = "../data/IMDB-Movie-Data.csv"
moviesDF: DataFrame = [Rank: int, Title: string ... 10 more fields]
rddMovies: RDD[Movie] = MapPartitionsRDD[13] at map at cmd5.sc:7

In [7]:
// Print the title of the first 10 movies to see if they were correctly added.
rddMovies.take(10).map(m => m.title).foreach(println)

take at cmd6.sc:1

1 / 1

Guardians of the Galaxy
Prometheus
Split
Sing
Suicide Squad
The Great Wall
La La Land
Mindhorn
The Lost City of Z
Passengers


## Part 1 - Playing with the movies using RDD functions

The goal of this part is to play (i.e. query, filter and transform the data) with the movies.

### Ex1 - Print the movies whose title contains "City" 

Goal: 

* use `map()` and `filter()` methods to get the title of the movies that contains "City" in their title
 
Output example:

```plain
City of Tiny Lights
The Mortal Instruments: City of Bones
```

Steps:

* Use `filter()` to only keep the movies that contains "City" in their title
* Use `map()` to retrieve the titles of these filtered movies
* Use `foreach()` to pretty print the results


In [8]:
// TODO students
rddMovies.map(m => m.title).filter(x => x.contains("City")).foreach(println)

foreach at cmd7.sc:1

1 / 1

The Lost City of Z
Sin City: A Dame to Kill For
City of Tiny Lights
The Mortal Instruments: City of Bones
Sex and the City
Sex and the City 2


### Ex2- Print the title of the movies rated between `rateMin` and `rateMax`. Take the 10 worst ratings.

Goal:
    
* Take the titles of the movies that were rated between `rateMin` and `rateMax` (exclusing `rateMin` and including`rateMax`).
* This list is sorted by rating ASC
    
Output example:

```plain
...
3.5 - Wrecker
3.7 - The Last Face
...
```
    
Steps:

* Use `filter()` to only keep the movies released between `rateMin` and `rateMax`
* Sort the filtered movies by decreasing rating
* Use `map()` to keep only the relevant attributes (i.e. rating and title)
* Use `foreach()` to pretty print the results

In [9]:
// TODO student
val rateMin = 3.5
val rateMax = 4.1
rddMovies
    .filter(m => rateMin < m.rating && m.rating <= rateMax)
    .sortBy(m => m.rating)
    .map(m => (m.rating, m.title))
    .foreach(m => println(m._1 + " - " + m._2))

sortBy at cmd8.sc:5

1 / 1

foreach at cmd8.sc:7

1 / 1

3.7 - The Last Face
3.7 - Satanic
3.9 - The Disappointments Room
3.9 - The Black Room
3.9 - Birth of the Dragon
4.0 - 2307: Winter's Dream
4.1 - Fifty Shades of Grey


rateMin: Double = 3.5
rateMax: Double = 4.1

### Ex3 - Print the 10 top genres

Goals:

* Print the list of the genres that appears the most
* Use `flatMap()`

Output example:

```plain
Drama (513)
Action (303)
Comedy (279)
Adventure (259)
```

Theory:

When an operation is giving you a sequence of sequences like:

```scala
Array("hello", "world").map(word => word.split(""))
res91: Array[Array[String]] = Array(Array(h, e, l, l, o), Array(w, o, r, l, d))
```

You may want to flatten this to only have a single list like:
```scala
Array("hello", "world").map(_.split("")).flatten
res93: Array[String] = Array(h, e, l, l, o, w, o, r, l, d)
```

You can achieve the same result (i.e. `map` + `flatten`) using `flatMap`:
```scala
Array("hello", "world").flatMap(_.split(""))
res95: Array[String] = Array(h, e, l, l, o, w, o, r, l, d)
```

We are going to apply this same technique with the `genres` member.

Steps:

* Use `flatMap()` to get the list with all the genres
* Make sure to remove trailling whitespaces
* Count the genres
* Sort them by decreasing order
* Show the top N genres

In [10]:
// TODO student
val N = 5
rddMovies
    .flatMap(m => m.genres)
    .groupBy(w => w)
    .map(element => (element._1, element._2.size))
    .sortBy(genre => genre._2, ascending=false)
    .take(N)
    .foreach(genre => println(genre._1 + "(" + genre._2 + ")"))

groupBy at cmd9.sc:4

1 / 1

sortBy at cmd9.sc:6

1 / 1

take at cmd9.sc:7

1 / 1

Drama(513)
Action(303)
Comedy(279)
Adventure(259)
Thriller(195)


N: Int = 5

### Ex4 - Print the average number of votes per year, order by descreasing number of votes

Goal:

* Print the average votes per year
* This output is sorted by descreasing votes

Output example:

```plain
...
year: 2008 average votes: 275505.3846153846
year: 2009 average votes: 255780.64705882352
year: 2010 average votes: 252782.31666666668
...
```

Theory:

We are going to use `reduceByKey()` which has the following signature `reduceByKey(func: (V, V) => V): RDD[(K, V)]`. 

`reduceByKey()` works on a RDD like `RDD[(K,V)]` (i.e. sort of "list of key/values pairs"). 

`reduceByKey()` takes a function that, from two elements, returns one i.e. the `func: (V, V) => V` in the signature.
The difference with `reduce()` is that `reduceByKey()` uses two elements sharing the same key.

For example (pseudo code):

```plain
 year, count
(2010, 2)
(2011, 3)
(2011, 4)
(2010, 8)
// use reduceByKey((count1, count2) => count1+count2)
> (2010, 10)
> (2011, 7)
```

Note: here `count` is just an Int but it can be anything e.g. `Movie`

Steps:

* To compute the average we need the **total sum** of votes per year and the **count** of all the movies per year
* Use `map()` to create an RDD made of `(year, (votes, 1))`. Like a word count we use the `1` to be able to count the number of movies per year
* Use `reduceByKey()` to sum the votes and to count the number of movies per year. The output should look like: `(year, (totalVotes, moviePerYearCount))`
* Find a way to compute the average using the result from the last operation
* Sort by number of votes decreasing

In [11]:
// TODO student
val base = rddMovies
    .map(m => (m.year, (m.votes, 1)))
    .reduceByKey((m1, m2) => (m1._1 + m2._1, m1._2 + m2._2))
    .map(elem => (elem._1, elem._2._1 / (elem._2._2 * 1.0)))
    .sortBy(elem => elem._2, ascending=false)
    .foreach(elem => println("year: " + elem._1 + " average votes: " + elem._2))

map at cmd10.sc:2

1 / 1

sortBy at cmd10.sc:5

1 / 1

foreach at cmd10.sc:6

1 / 1

year: 2012 average votes: 285226.09375
year: 2008 average votes: 275505.3846153846
year: 2006 average votes: 269289.95454545453
year: 2009 average votes: 255780.64705882352
year: 2010 average votes: 252782.31666666668
year: 2007 average votes: 244331.03773584907
year: 2011 average votes: 240790.3015873016
year: 2013 average votes: 219049.64835164836
year: 2014 average votes: 203930.22448979592
year: 2015 average votes: 115726.22047244094
year: 2016 average votes: 48591.75420875421


## Part 2 - Create a basic Inverted Index

The goal of this part is to show you how to create an inverted index that indexes words from all the movies' description.

Goal:

Using `rddMovies` create an inverted that use the movies' description:

```plain
Movie(1,Guardians of the Galaxy,List(Action, Adventure, Sci-Fi),A group of intergalactic [...] of the universe.,James Gunn,List(Chris Pratt, Vin Diesel, Bradley Cooper, Zoe Saldana),2014,8.1,757074.0)
Movie(2,Prometheus,List(Adventure, Mystery, Sci-Fi),Following clues to the origin[...] not alone.,Ridley Scott,List(Noomi Rapace, Logan Marshall-Green, Michael Fassbender, Charlize Theron),2012,7.0,485820.0)
Movie(3,Split,List(Horror, Thriller),Three girls are kidnapped [...] a frightful new 24th.,M. Night Shyamalan,List(James McAvoy, Anya Taylor-Joy, Haley Lu Richardson, Jessica Sula),2016,7.3,157606.0)
...
```

and extract them to produce an inverted index like:

```plain
"reunion" -> (640, 697)
"runner" -> (338)
"vietnam" -> (797, 947, 983)
...
```

Steps

* Tokenize description i.e. produce an RDD like (movId, words)
* Normalize words e.g. toLowercase, trimming,..
* Remove stopwords (ignored here)
* Apply stemming (ignored here)
* Group by document id

In [12]:
    /**
    * Goal: create an inverted index that allows searching a word
    * in the movies description.
    * Features:
    * - case insensitive
    *
    */
    // TODO student
    // In this first function we are going to tokenize and order the descriptions of the movies, then return these data. We are not going to apply any search right now.
    def createInvertedIndex(movies: RDD[Movie]): RDD[(String, Iterable[Int])] = {
        // Define helper functions directly inside this function. In scala you can declare inner functions
        // and use them only inside the function they were declared. Useful to encapsulate/restrict 
        // their use outside this function.
        
        // Split the given string into an array of words (without any formatting), then return it.
        def tokenizeDescription(description: String): Seq[String] = {
            return description.split(" ")
        }
        
        // Remove the blank spaces (trim) in the given word, transform it in lowercase, then return it.
        def normalizeWord(word: String): String = {
            return word.trim().toLowerCase()
        }
        
        // For the sake of simplicity let's ignore the implementation (in a real case we would return true if w is a stopword, otherwise false).
        // TODO student nothing here but still call this function in your invertedIndex creation process.
        def isStopWord(w: String): Boolean = {
          false
        }
        
        // For the sake of simplicity let's ignore the implementation (in a real case we would apply stemming to w and return the result, e.g. w=automation -> w=automat).
        // TODO student nothing here but still call this function in your invertedIndex creation process.
        def applyStemming(w: String): String = {
          w
        }
      
       // TODO student
       // Here we are going to work on the movies RDD, by tokenizing and normalizing the description of every movie, then by building a key-value object 
       // that contains the tokens as keys, and the IDs of the movies as values
       // (see the example on 4).
       // The goal here is to do everything by chaining the possible transformations and actions of Spark.
       // Possible steps:
       //   1) What we first want to do here is applying the 4 previous methods on any movie's description. Be aware of the fact that we also want to 
       //       keep the IDs of the movies.
       //   2) For each tokenized word, create a tuple as (word, id), where id is the current movie id
       //        [
       //          ("toto", 120), ("mange", 120), ("des", 120), ("pommes", 120),
       //          ("toto", 121), ("lance", 121), ("des", 121), ("photocopies", 121)
       //        ]
       //      Hint: you can use a `map` function inside another `map` function.
       //   3) We finally need to find a way to remove duplicated keys and thus only having one entry per key, with all the linked IDs as values. For example:
       //        [
       //          ("toto", [120, 121]),
       //          ("mange", [120]),
       //          ...
       //        ]
       val invertedIndex = movies
                            .flatMap(movie => tokenizeDescription(movie.description)
                                        .map(word => (normalizeWord(word), movie.id)))
                            .filter(movie => !isStopWord(movie._1))
                            .map(movie => (applyStemming(movie._1), movie._2))
                            .groupByKey()

                            
                            
                            
        
       // Return the new-built inverted index.
       invertedIndex
  }

defined function createInvertedIndex

Now we would like to use our inverted index to display the top N most used words in the descriptions of movies.

In [13]:
// TODO student
// Here we are going to operate the analytic and display its result on a given inverted index (that will be obtained from the previous function).
def topN(invertedIndex: RDD[(String, Iterable[Int])], N: Int): Unit = {
  // TODO student
  // We are going to work on the given invertedIndex array to do our analytic:
  //   1) Find a way to get the number of movie in which a word appears.
  //   2) Keep only the top N words and their occurence.
  val topMovies = invertedIndex
                    .map(movie => (movie._1, movie._2.size))
                    .sortBy(movie => movie._2, ascending=false)
                    .take(N)
  
  // Print the words and the number of descriptions in which they appear.
  println("Top '" + N + "' most used words")
  topMovies.foreach(println)
}

defined function topN

In [14]:
// Code used to test your implementation.
// Create the inverted index of the movies.
val invertedIndex = createInvertedIndex(rddMovies)

// Show how the inverted index looks like.
invertedIndex.take(3).foreach(x => println(x._1 + ": " + x._2.mkString(", ")))

// Show the top 10 most used words.
topN(invertedIndex, 10)

map at cmd11.sc:52

1 / 1

take at cmd13.sc:4

1 / 1

avatars,: 582
divorcee: 63
park,: 86, 86


...

1 / 1

take at cmd12.sc:9

1 / 1

Top '10' most used words
(a,1626)
(the,1357)
(to,933)
(of,806)
(and,713)
(in,573)
(his,485)
(an,304)
(is,294)
(with,274)


invertedIndex: RDD[(String, Iterable[Int])] = ShuffledRDD[37] at groupByKey at cmd11.sc:52

## Part 3 - Dataframe and SparkSQL

For all of the following exercices, write your queries in two different ways: 

* using the sql literal 
* using DataFrame API (select, where, etc.)

### Exercice 1 - Use the moviesDF DataFrame

* Use the dataframe `moviesDF` already created when loading the data 
* Print the schema of moviesDF
* Show the first 10 lines of the moviesDF as a table

In [15]:
// TODO students
moviesDF.printSchema()
// je select que les titres pour une meilleure lisibilité
val moviesTen = moviesDF.select("Title").limit(10)
moviesTen.show()


root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Runtime (Minutes): integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Votes: integer (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: integer (nullable = true)



show at cmd14.sc:4

1 / 1

+--------------------+
|               Title|
+--------------------+
|Guardians of the ...|
|          Prometheus|
|               Split|
|                Sing|
|       Suicide Squad|
|      The Great Wall|
|          La La Land|
|            Mindhorn|
|  The Lost City of Z|
|          Passengers|
+--------------------+



moviesTen: Dataset[Row] = [Title: string]

### Exercice 2 - Get the movies (id, title, votes, director) whose title contains "City" 

Apply two different ways: 

* use the sql literal 
* use DataFrame API (select, where, etc.)


In [16]:
// TODO students
moviesDF.createOrReplaceTempView("movies")
val cityMoviesSql = spark.sql("""SELECT rank, title, votes, director FROM movies WHERE title LIKE '%City%' """)
cityMoviesSql.show()

val cityMoviesDFApi = moviesDF.select("rank", "title", "votes", "director").where("title like '%City%'")
cityMoviesDFApi.show()

show at cmd15.sc:3

1 / 1

+----+--------------------+------+--------------------+
|rank|               title| votes|            director|
+----+--------------------+------+--------------------+
|   9|  The Lost City of Z|  7188|          James Gray|
| 427|Sin City: A Dame ...|122185|        Frank Miller|
| 506| City of Tiny Lights|   291|         Pete Travis|
| 684|The Mortal Instru...|112313|        Harald Zwart|
| 894|    Sex and the City|102547|Michael Patrick King|
| 953|  Sex and the City 2| 62403|Michael Patrick King|
+----+--------------------+------+--------------------+



show at cmd15.sc:6

1 / 1

+----+--------------------+------+--------------------+
|rank|               title| votes|            director|
+----+--------------------+------+--------------------+
|   9|  The Lost City of Z|  7188|          James Gray|
| 427|Sin City: A Dame ...|122185|        Frank Miller|
| 506| City of Tiny Lights|   291|         Pete Travis|
| 684|The Mortal Instru...|112313|        Harald Zwart|
| 894|    Sex and the City|102547|Michael Patrick King|
| 953|  Sex and the City 2| 62403|Michael Patrick King|
+----+--------------------+------+--------------------+



cityMoviesSql: DataFrame = [rank: int, title: string ... 2 more fields]
cityMoviesDFApi: Dataset[Row] = [rank: int, title: string ... 2 more fields]

### Exercice 3 - Get the number of movies which have a number of votes between 500 and 2000 (inclusive range)

In [17]:
// TODO students
moviesDF.createOrReplaceTempView("movies")
val sql3 = spark.sql("""select count(*) as count
                        from movies
                        where votes >= 500
                        and votes <= 2000""")
sql3.show()

val dFApi3 = moviesDF.select("*").where("votes >= 500 and votes <= 2000").count()
println(dFApi3)

show at cmd16.sc:6

1 / 1

show at cmd16.sc:6

1 / 1

+-----+
|count|
+-----+
|   34|
+-----+



...

1 / 1

count at cmd16.sc:8

1 / 1

34


sql3: DataFrame = [count: bigint]
dFApi3: Long = 34L

### Exercice 4 - Get the minimum, maximum and average rating of films per director. Sort the results by minimum rating.  

In [18]:
// TODO students
// résultats un peu différent il semblerait que l'ordre lors du tri au sein d'une même valeur ne soit pas le même
moviesDF.createOrReplaceTempView("movies")
val sql4 = spark.sql("""select director, min(rating) as min, max(rating), avg(rating) from movies group by director order by min""")

sql4.show()

val dFApi4 = moviesDF.groupBy("director").agg(min(col("rating")) as "min", max(col("rating")), avg(col("rating"))).orderBy(col("min"))
dFApi4.show()

show at cmd17.sc:4

1 / 1

show at cmd17.sc:4

200 / 200

+--------------------+---+-----------+-----------------+
|            director|min|max(rating)|      avg(rating)|
+--------------------+---+-----------+-----------------+
|     Jason Friedberg|1.9|        1.9|              1.9|
|       Shawn Burkett|2.7|        2.7|              2.7|
|          James Wong|2.7|        2.7|              2.7|
|   Jonathan Holbrook|3.2|        3.2|              3.2|
|       Femi Oyeniran|3.5|        3.5|              3.5|
|      Micheal Bafaro|3.5|        3.5|              3.5|
|           Sean Penn|3.7|        8.1|              5.9|
|     Jeffrey G. Hunt|3.7|        3.7|              3.7|
|         D.J. Caruso|3.9|        6.9|            5.875|
|        George Nolfi|3.9|        7.1|              5.5|
|      Rolfe Kanefsky|3.9|        3.9|              3.9|
|         Joey Curtis|4.0|        4.0|              4.0|
|  Sam Taylor-Johnson|4.1|        4.1|              4.1|
|  M. Night Shyamalan|4.2|        7.3|5.533333333333332|
|    Gee Malik Linton|4.2|     

...

1 / 1

show at cmd17.sc:7

200 / 200

+--------------------+---+-----------+-----------------+
|            director|min|max(rating)|      avg(rating)|
+--------------------+---+-----------+-----------------+
|     Jason Friedberg|1.9|        1.9|              1.9|
|       Shawn Burkett|2.7|        2.7|              2.7|
|          James Wong|2.7|        2.7|              2.7|
|   Jonathan Holbrook|3.2|        3.2|              3.2|
|      Micheal Bafaro|3.5|        3.5|              3.5|
|       Femi Oyeniran|3.5|        3.5|              3.5|
|           Sean Penn|3.7|        8.1|              5.9|
|     Jeffrey G. Hunt|3.7|        3.7|              3.7|
|        George Nolfi|3.9|        7.1|              5.5|
|      Rolfe Kanefsky|3.9|        3.9|              3.9|
|         D.J. Caruso|3.9|        6.9|            5.875|
|         Joey Curtis|4.0|        4.0|              4.0|
|  Sam Taylor-Johnson|4.1|        4.1|              4.1|
|  M. Night Shyamalan|4.2|        7.3|5.533333333333332|
|    Gee Malik Linton|4.2|     

sql4: DataFrame = [director: string, min: double ... 2 more fields]
dFApi4: Dataset[Row] = [director: string, min: double ... 2 more fields]

### Exercice 5 - Find the title of the movie(s) having the minimum rating for each year. Show the title, year and the rating information in the result, order by increasing rating.

**Example output**

```plain
+--------------------+----+------+
|               title|year|rating|
+--------------------+----+------+
|      Disaster Movie|2008|   1.9|
|Don't Fuck in the...|2016|   2.7|
|Dragonball Evolution|2009|   2.7|
...
```

In [21]:
// TODO students
moviesDF.createOrReplaceTempView("movies")
val sql5 = spark.sql("select m1.title, m1.year, m1.rating from movies m1 join (select year, min(rating) as min from movies group by year) m2 on m2.year = m1.year and m2.min = m1.rating order by m1.year")
sql5.show()

// comment empêcher les duplicatats de colonnes?
val dFtry = moviesDF.select("title", "rating", "year")
val dFmin = moviesDF.groupBy(col("year").alias("minYear")).agg(min(col("rating")).alias("minRating"))
val dFApi5 = dFtry.join(dFmin, dFmin("minYear") === dFtry("year") && dFmin("minRating") === dFtry("rating"))
dFApi5.show()

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

200 / 200

show at cmd20.sc:3

...

200 / 200

show at cmd20.sc:8

1 / 1

+--------------------+------+----+-------+---------+
|               title|rating|year|minYear|minRating|
+--------------------+------+----+-------+---------+
|Don't Fuck in the...|   2.7|2016|   2016|      2.7|
|Aliens vs Predato...|   4.7|2007|   2007|      4.7|
|     Spring Breakers|   5.3|2012|   2012|      5.3|
|            Movie 43|   4.3|2013|   2013|      4.3|
|  The Last Airbender|   4.2|2010|   2010|      4.2|
|            Sex Tape|   5.1|2014|   2014|      5.1|
|    Megan Is Missing|   4.9|2011|   2011|      4.9|
|   Lady in the Water|   5.6|2006|   2006|      5.6|
|      Disaster Movie|   1.9|2008|   2008|      1.9|
|Dragonball Evolution|   2.7|2009|   2009|      2.7|
|   Snakes on a Plane|   5.6|2006|   2006|      5.6|
|The Twilight Saga...|   4.9|2011|   2011|      4.9|
|             Wrecker|   3.5|2015|   2015|      3.5|
+--------------------+------+----+-------+---------+



sql5: DataFrame = [title: string, year: int ... 1 more field]
dFtry: DataFrame = [title: string, rating: double ... 1 more field]
dFmin: DataFrame = [minYear: int, minRating: double]
dFApi5: DataFrame = [title: string, rating: double ... 3 more fields]

### Exercice 6 - Find the title of movies having the same director. 

**Example output**

```plain
+------------------+--------------------+--------------------+
|         director1|              title1|              title2|
+------------------+--------------------+--------------------+
|        James Gunn|Guardians of the ...|               Super|
|        James Gunn|Guardians of the ...|             Slither|
|      Ridley Scott|          Prometheus|        Body of Lies|
|      Ridley Scott|          Prometheus|         A Good Year|
...
```

Note that when using the dataframe API:
* You can use `===` to test equality of columns and `!==` to test non-equality of them
* You can use the `as` keyword to make alias of column names and table names (check the reference documentation)

In [ ]:
// TODO students